# 学習型射影勾配法


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/wadayama/MIKA2019/blob/master/ProjectedGradient.ipynb)

学習型射影勾配法のサンプルコードです。ステップサイズを固定した方式と比較しています。

## 必要なパッケージのインポート

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import math
import matplotlib.pyplot as plt

## グローバル定数の設定

In [ ]:
n = 500 # 信号長
mbs = 100 # ミニバッチサイズ
noise_std = 0.5 # 雑音の標準偏差
alpha = 8.0 # 射影関数のソフトさを表す
A = torch.normal(mean=torch.zeros(n,n), std = 1.0)
max_itr = 10 # 射影勾配法の反復回数
adam_lr = 0.00005 # Adamの学習率

## ミニバッチ生成関数

In [ ]:
def gen_minibatch():
    x = 1.0 - 2.0 * torch.randint(0, 2, (mbs, n)).float()
    w = torch.normal(mean=torch.zeros(mbs, n), std = noise_std)
    y = torch.mm(x, A.t()) + w
    return x, y

## 学習型射影勾配法 TPG クラス (Trainable Projected Gradient)

In [ ]:
class TPG(nn.Module):
    def __init__(self):
        super(TPG, self).__init__()
        self.beta = nn.Parameter(0.01*torch.ones(max_itr)) # 学習可能ステップサイズパラメータ
    def forward(self, y, num_itr):
        s = torch.zeros(mbs, n) # 初期探索点
        for i in range(num_itr):
            r = s + self.beta[i] * torch.mm(y - torch.mm(s, A.t()), A) # 勾配ステップ
            s = torch.tanh(alpha * r) # 射影ステップ
        return s

## 訓練ループ(インクリメンタルトレーニング）

In [ ]:
model = TPG()
opt   = optim.Adam(model.parameters(), lr=adam_lr)
loss_func = nn.MSELoss()
for gen in range(max_itr):
    for i in range(100):
        x, y = gen_minibatch()
        opt.zero_grad()
        x_hat = model(y, gen + 1)
        loss  = loss_func(x_hat, x)
        loss.backward()
        opt.step()
    print(gen, loss.item())

## 学習しない射影勾配法

In [ ]:
class PG(nn.Module): # 普通の射影勾配法
    def __init__(self):
        super(PG, self).__init__()
    def forward(self, y, num_itr, gamma):
        s = torch.zeros(mbs, n) # 初期探索点
        for i in range(num_itr):
            r = s + gamma * torch.mm(y - torch.mm(s, A.t()), A)  # 勾配ステップ　
            s = torch.tanh(alpha * r) # 射影ステップ
        return s
pg_model = PG()

## 誤差のプロット

In [ ]:
mbs = 1
x, y = gen_minibatch()

norm_list = []
itr_list = []

with torch.no_grad():
    for i in range(max_itr):
        err = (torch.norm(x - model(y, i))**2).item()/n
        norm_list.append(10.0*math.log10(err))
        itr_list.append(i)
plt.plot(itr_list, norm_list, color="red", label="Trainable-PG")

norm_list = []
itr_list = []

for i in range(max_itr):
    err = (torch.norm(x - pg_model(y, i, 0.0010))**2).item()/n
    norm_list.append(10.0*math.log10(err))
    itr_list.append(i)
plt.plot(itr_list, norm_list, color="blue", label="PG-1")

norm_list = []
itr_list = []
for i in range(max_itr):
    err = (torch.norm(x - pg_model(y, i, 0.0015))**2).item()/n
    norm_list.append(10.*math.log10(err))
    itr_list.append(i)
plt.plot(itr_list, norm_list, color="black", label="PG-2")

norm_list = []
itr_list = []
for i in range(max_itr):
    err = (torch.norm(x - pg_model(y, i, 0.0005))**2).item()/n
    norm_list.append(10.0*math.log10(err))
    itr_list.append(i)
plt.plot(itr_list, norm_list, color="green", label="PG-3")

plt.grid()
plt.xlabel("iteration")
plt.ylabel("squared error (dB)")
plt.legend()
plt.ylim([-30, 5])
plt.show()

In [ ]:
print('beta = ', model.beta)